In [3]:
#Import libraries
import pandas as pd
import re 

In [189]:
dirPath = r"C:\Study\Projects\NLP\data"
#reviewPath = r"reviews_Tools_and_Home_Improvement_5.json.gz"
reviewPath = r"reviews_Musical_Instruments_5.json.gz"
qaPath = r"C:\Study\Projects\NLP\data" 
path = dirPath + "\\" + reviewPath
path = "preprocessed_reviews_Musical_Instruments_5.pkl.gz"
path = "preprocessed_reviews_Tools_and_Home_Improvement_5.pkl.gz"

In [190]:
data = pd.read_pickle(path)

In [191]:
data.head(5)

,masterIndex,asin,reviewText,helpful,overall,trueRating,CleanedJoin
0,0,104800001X,"I hate it when my shirt collars, not otherwise...","[0, 1]",5.0,1.0,i hate it when my shirt collar not otherwise s...
1,1,104800001X,These little magnets are really powerful for t...,"[0, 0]",5.0,1.0,these little magnet be really powerful for the...
2,2,104800001X,I wanted something this small to mount on the ...,"[0, 0]",5.0,1.0,i want something this small to mount on the ba...
3,3,104800001X,I use these to magnetize my Warhammer 40K mini...,"[0, 0]",5.0,1.0,i use these to magnetize my warhammer k miniat...
4,4,104800001X,They are soo freaking annoying!! Why?! You sp...,"[23, 25]",5.0,1.0,"they be soo freak annoy, why, you spend all th..."


In [192]:
import itertools

largeText = data.CleanedJoin.to_list()
#largeText = reviewTextProcessing.reviewText.to_list() # Not too good accuracy around 116 missed with %86.94

len(largeText)

123707

In [193]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import confusion_matrix

In [194]:
X = largeText
y = data['trueRating']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=310)

x_train_text = X_train
x_test_text = X_test

In [195]:
len(y_train), len(y_test), len(x_train_text), len(x_test_text)

(92780, 30927, 92780, 30927)

In [211]:
base = 500
factor = 40
top_num_of_words = base * factor
#vect = TfidfVectorizer(max_features=top_num_of_words, stop_words = 'english')
vect = TfidfVectorizer(max_features=top_num_of_words, stop_words = 'english', ngram_range=(1,2), max_df=0.9, min_df=2, use_idf=True, sublinear_tf=True)
#vect = TfidfVectorizer(max_features=top_num_of_words, ngram_range=(1, 2), max_df=0.9, min_df=2, use_idf=True, sublinear_tf=True)


In [212]:
%%time
text = x_train_text
vect.fit(text)
tfidf_train = vect.transform(text)


Wall time: 36.3 s


In [213]:
model = MultinomialNB()

In [214]:
#model = LogisticRegression(C=0.2, dual=True)
#(0, 2418, 0, 31201) nooo negative

In [215]:
MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [216]:
index = len(x_train_text)
index

92780

In [217]:
%%time
model.fit(tfidf_train, y_train)
#model.fit(x_train_pad, y_train,validation_split=0.05, epochs=3, batch_size=74)

Wall time: 63 ms


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [218]:
%%time
vect.fit(x_test_text)

Wall time: 8.65 s


TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=0.9, max_features=20000,
                min_df=2, ngram_range=(1, 2), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words='english', strip_accents=None,
                sublinear_tf=True, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [219]:
tfidf_test = vect.transform(x_test_text)

In [220]:
vocab = vect.get_feature_names()
len(vocab), vocab[1:5]

(20000, ['aa aaa', 'aa battery', 'aa cell', 'aa flashlight'])

In [221]:
tfidf_test

<30927x20000 sparse matrix of type '<class 'numpy.float64'>'
	with 1427199 stored elements in Compressed Sparse Row format>

In [222]:
y_preds = model.predict(tfidf_test)
accuracy = (y_preds==y_test).mean()


In [223]:
print("Accuracy: {0:.2%}".format(accuracy))


Accuracy: 79.79%


In [224]:
tn, fp, fn, tp = confusion_matrix(y_test, y_preds).ravel()
tn, fp, fn, tp

(509, 2074, 4175, 24169)

In [210]:
pd.DataFrame(y_preds).describe()

,0
count,30927.000000
mean,0.999677
std,0.017979
min,0.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,1.000000


# 

In [ ]:
pd.options.display.max_colwidth = 500

#productsReviewData.cleanedText.head(2)